$$
% transferring macros:
\let\eps\varepsilon
\let\wt\widetilde
\let\tn\mathrm
\let\vphi\varphi
\let\pa\partial
\let\para\parallel
\let\wh\widehat
\let\sl\shoveleft
\let\lra\leftrightarrow
\let\ov\overline
% defining new macros:
\def\be{\begin{equation} \mskip 1cm}
\def\ee{\end{equation}}
\def\ba{\begin{align} \mskip 1cm}
\def\ea{\end{align}}
\def\RR{{\mathbb R}}
\def\DD{{\mathbb D}}
\def\EE{{\mathbb E}}
\def\GG{{\mathbb G}}
\def\CC{{\mathbb C}}
\def\MM{{\mathbb M}}
\def\PP{{\mathbb P}}
\def\JJ{{\mathbb J}}
\def\WW{{\mathbb W}}
\def\HH{{\mathbb H}}
\def\SS{{\mathbb S}}
\def\QQ{{\mathbb Q}}
\def\Bb{{\mathbf B}}
\def\Eb{{\mathbf E}}
\def\Fb{{\mathbf F}}
\def\Gb{{\mathbf G}}
\def\Ub{{\mathbf U}}
\def\Vb{{\mathbf V}}
\def\Ab{{\mathbf A}}
\def\Jb{{\mathbf J}}
\def\xb{{\mathbf x}}
\def\cb{{\mathbf c}}
\def\eb{{\mathbf e}}
\def\nb{{\mathbf n}}
\def\wb{{\mathbf w}}
\def\ub{{\mathbf u}}
\def\ab{{\mathbf a}}
\def\bb{{\mathbf b}}
\def\rb{{\mathbf r}}
\def\vb{{\mathbf v}}
\def\ib{{\mathbf i}}
\def\jb{{\mathbf j}}
\def\kb{{\mathbf k}}
\def\pb{{\mathbf p}}
\def\qb{{\mathbf q}}
\def\zb{{\mathbf z}}
\def\Fb{{\mathbf F}}
\def\Ib{{\mathbf I}}
\def\Cb{{\mathbf C}}
\def\Yb{{\mathbf Y}}
\def\Mb{{\mathbf M}}
\def\Mcal{\mathcal M}
\def\Vcal{\mathcal V}
\def\Wcal{\mathcal W}
\def\Gcal{\mathcal G}
\def\Bcal{\mathcal B}
\def\Dcal{\mathcal D}
\def\Ncal{\mathcal N}
\def\vs{\mathsf v}
\def\as{\mathsf a}
\def\grad{\tn{grad}}
\def\curl{\tn{curl}}
\def\unit{\mathbb 1}
\def\div{\tn{div}}
\def\Im{\tn{Im}}
\def\vol{\tn{vol}}
\def\dd{\tn{d}}
\def\ii{\tn{i}}
\def\phib{\boldsymbol \phi}
\def\psib{\boldsymbol \psi}
\def\rhob{\boldsymbol \rho}
\def\Lamb{\boldsymbol \Lambda}
\def\ts{\tn s}
\def\tA{\tn A}
\def\tc{\tn c}
\def\ti{\tn i}
\def\te{\tn e}
\def\th{\tn h}
\def\tt{\tn t}
\def\tg{\tn g}
\def\tE{\tn E}
\def\tB{\tn B}
\def\td{\tn d}
\def\tb{\tn b}
\newcommand{\fhD}{f_\tn{h}^\tn{D}}
\newcommand{\whD}{w_\tn{h}^\tn{D}}
\newcommand{\nhD}{n_\tn{h}^\tn{D}}
\newcommand{\jhD}{\jb_\tn{h}^\tn{D}}
% defining new commands:
\newcommand{\abs}[1]{\lvert#1\rvert}
\newcommand{\bold}[1]{{\boldsymbol #1}}
\newcommand{\pder}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\parfra}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\dt}[1]{\frac{\mathrm d #1}{\mathrm dt}}
\newcommand{\ds}[1]{\frac{\mathrm d #1}{\mathrm ds}}
\newcommand{\gavg}[1]{\left\langle #1 \right\rangle}
\def\dd{\mathrm{d}}
\def\pab{\bold \pa}
\newcommand{\etab}{\boldsymbol{\eta}}
\newcommand{\xib}{\boldsymbol{\xi}}
\newcommand{\gavg}[1]{\langle #1 \rangle}
$$

# 1D Vlasov-Ampère with STRUPHY routines

Written by Stefan Possanner (<stefan.possanner@ma.tum.de>), May 2020.

* [The model](#model)
* [Particle-in-cell (PIC) approach](#pic)
* [Logical coordinates](#logic)
    * [Ampère equation](#ampere)
    * [Vlasov equation](#vlasov)
    * [Expectation value](#expect)
* [Numerical scheme](#scheme)
    * [Semi-discrete in space (continuous in time)](#semi)
    * [Conservation of Gauss' law and initialization](#gauss)
    * [Time discretization](#time)

## The model <a class="anchor" id="model"></a>

A common academic test model for particle-in-cell (PIC) codes is the 1D Vlasov-Poisson system:

$$ 
\be 
\left\{
\begin{aligned}
 &\pa_t\, f + v\,\pa_x\, f - E\,\pa_v\, f = 0\,, \qquad f(0,x,v) = f_0(x,v)\,, \qquad && \tn{(Vlasov)}
 \\[1mm]
 &\pa_x E = 1 - \int_\RR f \,\tn d v\,. \qquad && \tn{(Poisson)}
\end{aligned}
\right.
\label{VP}
\ee
$$

Here, $f(t,x,v) > 0$ is the distribution function on $\RR \times \Omega$, where $\Omega = [0, L]\times \RR$ is the $(x,v)$-phase space, and $E(t,x) \in \RR$ denotes the electric field on $\RR \times [0,L]$. We assume the solution $(f,E)$ to be $L$-periodic in $x$. The ion density has been set to 1 and is thus constant in time.

In this notebook, rather than solving directly system \eqref{VP}, we replace Poisson's equation by Ampère's law in the absence of magnetic fields (all physical constants are set to 1, ion current is zero):

$$
\be
 - \pa_t E = -\int_\RR v f\,\tn d v\,.
\ee
$$

Taking the derivative with respect to $x$ and inserting the Vlasov equation yields

$$
\be
 - \pa_t \pa_x E = \pa_t \int_\RR f\,\tn d v \qquad \Leftrightarrow \qquad \pa_t \left( \pa_x E - 1 + \int_\RR f\,\tn d v\right) = 0\,.
\ee \label{Gauss:t}
$$

Therefore, the **Vlasov-Ampère** system

$$ 
\be 
\left\{
\begin{aligned}
 &\pa_t\, f + v\,\pa_x\, f - E\,\pa_v\, f = 0\,, \qquad f(0,x,v) = f_0(x,v)\,, \qquad && \tn{(Vlasov)}
 \\[1mm]
 &\pa_t E = \int_\RR v f\,\tn d v\,, \qquad \pa_x E(0,x) = 1 - \int_\RR f_0\,\tn d v\,, \qquad && \tn{(Ampère)}
\end{aligned}
\right.
\label{VA}
\ee
$$


is equivalent to system \eqref{VP}. Note that Gauss' law (Poisson equation) needs to be fulfilled at the initial time $t=0$; equation \eqref{Gauss:t} then guarantees that it stays fulfilled at all times, yielding the equivalence of \eqref{VP} and \eqref{VA}.

The energy conserved by the Vlasov-Ampère system reads

$$
\be
 H = \frac 12 \int_0^L\int_\RR |v|^2\,f\,\tn d v\,\tn d x + \frac 12 \int_0^L |E|^2\,\tn d x\,.
\ee \label{VA:en}
$$

This can be verified in a straightforward manner from the equations.

## Particle-in-cell (PIC) approach <a class="anchor" id="pic"></a>

The PIC method uses Lagrangian coordinates $(x_{k0},v_{k0}) \in \Omega$ to solve the Vlasov equation. More precisely, it consists of randomly drawing $K$ pairs of coordinates $(x_{k0},v_{k0})_{k=0}^{K-1} \in \Omega$ and solving for each pair the characteristic equations

$$
\be
\left\{
\begin{aligned}
 \dt{x_k(t)} &= v_k(t)\,,\qquad && x_k(0) = x_{k0}\,,
 \\[1mm]
 \dt{v_k(t)} &= - E(t,x_k(t)) \,,\qquad && v_k(0) = v_{k0}\,.
\end{aligned}
\right.
\ee \label{chars}
$$

This yields the Lagrangian paths $(x_k(t),v_k(t)) \in \Omega$. The Vlasov equation implies that $f$ is constant along these paths:

$$
\be
 \dt{}\, f(t,x_k(t),v_k(t)) = \pa_t\, f + v_k(t)\,\pa_x\, f - E(t,x_k(t))\,\pa_v\, f  = 0\,. 
\ee
$$

Therefore,

$$
\be
 f(t,x_k(t),v_k(t)) = f_0(x_{k0},v_{k0})\,.
\ee
$$

The coupling to Ampère's equation is interpreted as a **Monte-Carlo integral**. To see how this works, we need to write a weak formulation of Ampère, 

$$
\be
 \int_0^L \pa_t E\,Q\,\tn d x = \int_0^L\int_\RR v f\,Q\,\tn d v\,\tn d x\,\qquad \forall \ Q \in V\,,
\ee \label{weak}
$$

where $V$ is a suitable test function space. Suppose $s(t,x,v) > 0$ is a time-dependent probability density function (PDF), called the **sampling density**, that satisfies the Vlasov equation:

$$
\be
 \pa_t\, s + v\,\pa_x\, s - E\,\pa_v\, s = 0\,, \qquad s(0,x,v) = s_0(x,v)\,, \qquad \int_0^L \int_\RR s\,\tn d v \,\tn d x = 1\,.
\ee
$$

Then $s$ is also constant along the characteristics \eqref{chars}. The right-hand side of \eqref{weak} is re-written as

$$
\be
 \int_0^L\int_\RR v f\,Q\,\tn d v\,\tn d x = \int_0^L\int_\RR v\, \underbrace{\frac{f(t,x,v)}{s(t,x,v)}}_{=:w(t,x,v)}\,Q\,s(t,x,v)\,\tn d v\,\tn d x = \EE[V w(t,X,V) Q(X)]\,.
\ee \label{j}
$$

The last expression denotes the expectation value $\EE$ of the random variable $V w(t,X,V) Q(X)$, which is a function of the random variables $(X,V)$ for which we draw samples at the beginning. Here, $w(t,X,V)$ is the **weight function**. We have thus made the link to Monte-Carlo integration and probability theory. 

As a short reminder for the basics of Monte-Carlo, suppose that $Z$ stands for a continuous random variable, taking values in some real interval $[a,b]$ and distributed according to the PDF $s: [a,b]\to \RR_+$. Let us draw the $K$ samples $(z_k)_{k=0}^{K-1}$ distributed according to $s$. Moreover, let $h(Z)$ denote some function of the random variable, for instance the identity, $h(Z) = Z$. The expectation value of $h(Z)$ is defined as

$$
\be
 \EE[h(Z)] := \int_a^b h(z)\,s(z)\,\tn d z\,.
\ee
$$

From our samples we can compute an **estimator** $\hat\EE$ for the expectation value,

$$
\be
 \EE\approx \hat \EE := \frac 1K \sum_{k=0}^{K-1} h(z_k)\,.
\ee
$$

This estimator converges to the true expectation $\hat\EE \to \EE$ as $K\to\infty$, at the typical rate of $1/\sqrt K$ for Monte-Carlo integration. This is exactly how we approximate the expectation in \eqref{j}, namely

$$
\be
 \int_0^L\int_\RR v f\,Q\,\tn d v\,\tn d x = \EE[V w(X,V) Q(X)] \approx \hat \EE[V w(X,V) Q(X)] = \frac 1K \sum_{k=0}^{k-1} v_k(t)\, w_k(t)\,Q(x_k(t)) \,.
\ee 
$$

Finally, we can use that the weights are constant in time because $f$ and $s$ are constant along the characteristics:

$$
\be
 w_k(t) = \frac{f(t,x_k(t),v_k(t))}{s(t,x_k(t),v_k(t))} = \frac{f_0(x_{0k},v_{0k})}{s_0(x_{0k},v_{0k})} = w_{0k}\,,
\ee
$$

which yields 

$$
\be
 \int_0^L\int_\RR v f\,Q\,\tn d v\,\tn d x \approx \frac 1K \sum_{k=0}^{k-1}  w_{0k}\,v_k(t)\,Q(x_k(t)) \,.
\ee \label{j:MC}
$$

Here, the test function $Q$ needs to be evaluated at the current particle positions $x_k(t)$.

## Logical coordinates <a class="anchor" id="logic"></a>

### Ampère equation <a class="anchor" id="ampere"></a>

The Vlasov-Ampère system \eqref{VA} can be understood in the framework of differential forms used in STRUPHY as follows: In the Ampère equation, we shall use the 1D commuting diagram 

$$
\be
 \begin{matrix}
 H^1([0,1]) & \stackrel{\pa_x}{\to} & L^2([0,1])
 \\[2mm]
  \downarrow \Pi_0 & & \downarrow \Pi_1 
 \\
 V_0 & \stackrel{\pa_x}{\to} & V_1
 \end{matrix} \,.
\ee
$$

The space $H^1([0,1])$ contains the **coefficient functions** of 0-forms and the space $L^2([0,1])$ contains the coefficient functions of 1-forms (in 1D there are only 0-forms and 1-forms). In STRUPHY we solve Ampère (or field equations in general) on the **logical domain**, here $[0,1]$ and related to the physical domain $[0,L]$ via the mapping $F: \eta\mapsto x$. The scalar product in the two spaces read

$$
\be
 (\alpha^0,\,\beta^0) = \int_0^1 a\,b\,\sqrt{g}\,\tn d\eta\,, \qquad (\alpha^1,\,\beta^1) = \int_0^1 a_{1} b_{1}\,\frac{1}{\sqrt{g}}\,\tn d\eta\,,\qquad \sqrt g = \tn{det}\, DF\,. 
\ee
$$

Here, $DF$ is the Jacobian of the mapping $F$ and $a,b,a_1,b_1$ denote the coefficient functions of the forms $\alpha^0,\beta^0,\alpha^1,\beta^1$, respectively. We denote the correspondence by the left-right arrow $\lra$, e.g. $\alpha^0 \lra a$. The exterior derivative $\tn d \equiv \pa_\eta$ takes 0-forms to 1-forms. The Hodge-$*$ operator switches between forms (multiplication or division by the Jacobian determinant $\sqrt g$ of the mapping $F$):

$$
\begin{align}
 \alpha^0 \lra a \quad &\Leftrightarrow \quad  *\alpha^0 \lra \sqrt g\, a \quad \tn{(coeff.\ function\ of\ 1-form)} \,,
 \\[2mm]
 \alpha^1 \lra a_1 \quad &\Leftrightarrow \quad  *\alpha^1 \lra \frac{a_1}{\sqrt g}  \quad \tn{(coeff.\ function\ of\ 0-form)}\,.
\end{align}
$$

This shows that the Hodge-$*$ is an isometry, $(*\alpha^0,\,*\beta^0) = (\alpha^0,\,\beta^0)$ and $(*\alpha^1,\,*\beta^1) = (\alpha^1,\,\beta^1)$. Last but not least we need the co-differential (integration by parts) in 1D:

$$
\be
 (\tn d \alpha^0,\, \beta^1) = (-1) (\alpha^0,\, * \dd * \beta^1 )\,.
\ee
$$

On the logical domain, the weak form \eqref{weak} of Ampère's law reads

$$
\be
 \int_0^1 \pa_t E\,Q\,\sqrt g\,\tn d \eta = \int_0^1\int_\RR v f\,\tn d v\,Q\,\sqrt g\, \tn d \eta\,\qquad \forall \ Q \in V\,.
\ee \label{weak:logic}
$$

This can also be written as

$$
\be
 \int_0^1 \pa_t (\sqrt g E)\,(\sqrt g Q)\,\frac{1}{\sqrt g}\,\tn d \eta = \int_0^1\int_\RR v f\,\tn d v\,(\sqrt g Q)\,\tn d \eta\,\qquad \forall \ \sqrt Q \in V\,.
\ee \label{weak:logic:1}
$$

Here, $\sqrt g E =: E_1$ and $\sqrt g Q =: Q_1$ are the pullbacks of 1-forms (in 1D). Equation \eqref{weak:logic} corresponds to choosing $E^0,Q^0$ as 0-forms,

$$
\be
 (\pa_t E^0, Q^0) = \left( \int_\RR v (\sqrt g f)\,\tn d v, Q^0 \right)_{L^2}\,,\qquad \forall\ Q^0\,,
\ee \label{weak:0}
$$

whereas \eqref{weak:logic:1} corresponds to choosing $E^1,Q^1$ as 1-forms,

$$
\be
 (\pa_t E^1, Q^1) = \left( \int_\RR v f\,\tn d v, Q^1 \right)_{L^2}\,,\qquad \forall\ Q^1\,.
\ee \label{weak:1}
$$

Here, on the right-hand side stands the $L^2$ scalar product in the logical domain,

$$
\be
 (f,h)_{L^2} := \int_0^1 f(\eta)\,h(\eta)\,\tn d \eta\,.
\ee
$$



### Vlasov equation <a class="anchor" id="vlasov"></a>

As far as the **Vlasov equation in logical coordinates** is concerned, we need to recall some basic properties about transport equations: let us denote by $\etab \in \RR^{2n}$, $n\geq 1$, the phase space coordinates (position and velocity) of a given problem with particle equations of motion in the form

$$
\be
 \dt{\etab(t)} = \Gb(t,\etab(t))\,,\qquad \etab(0) = \etab_0 \in \RR^{2n}\,.
\ee \label{sys:etab}
$$

Here, $\Gb:\RR\times \RR^{2n} \to \RR^{2n}$ is called the _direction field_ of the ODE. Moreover, let us assume that \eqref{sys:etab} is a *Hamiltonian system* ($\Gb = J\nabla H$, with $J$ a Poisson matrix), and that the mapping $F:\etab \to \zb$ (time parameter $t$, Jacobian determinant $\sqrt g$) maps to canonical coordinates $\zb$, where the system reads $\dot \zb = J_0 \nabla H$ with $J_0$ the canonical Poisson matrix. We denote by $\Psi_t(\etab_0) = \etab(t)$ the _flow map_ of the ODE system, and by $D\Psi_t$ the corresponding Jacobian. By definition, the flow map satisfies $\dot \Psi_t = \Gb(t,\Psi_t)$.

There are two ways of introducing the distribution function $f(t,\etab)$, which can be shown to be equivalent:

1. **Conservation of particles:** for any volume $\Vcal_0 \in \RR^{2n}$ the total mass $M$ must be conserved under the flow $\Psi_t$:

$$
\be
  \dt{} \int_{\Psi_t(\Vcal_0)} f(t,\etab) \sqrt g(t,\etab)\,\tn d\etab = 0\,.
\ee \label{M:conserve}
$$

2. **Euler-Lagrange map:** the distribution function is linked to its initial condition via

$$
\be
 f(t,\etab) \sqrt g(t,\etab) = \int_{\RR^{2n}} \delta\big(\etab - \Psi_t(\etab_0) \big)\, f_0(\etab_0) \sqrt g(0,\etab_0)\,\tn d \etab_0\,,
\ee \label{ELmap}
$$

where $\delta(\etab)$ denotes the Dirac delta distribution.

Both definitions lead to the Vlasov equation in coordinates $\etab$. To show this we use the notation $f_1 = \sqrt g f$ for the **pseudo distribution function**, containing the Jacobian $\sqrt g$, and use the following important statements (without proof): 

- The Jacobian determinant of the flow satisfies

$$
\be
 \dt{} |\tn{det}(D\Psi_t)| = |\tn{det}(D\Psi_t)|\, \nabla \cdot \Gb(t,\Psi_t)\,,\qquad \tn{det}(D\Psi_t) = 1\,.
\ee \label{dot:DPsi}
$$

- The Liouville property of (non-canonical) Hamiltonian systems:

$$
\be
 \pa_t \sqrt g + \nabla \cdot (\sqrt g\, \Gb) = 0\,.
\ee \label{Liouville}
$$

Remark that the Jacobian of the flow $\tn{det}(D\Psi_t)$ is 1 at all times if $\nabla \cdot \Gb = 0$, which is the case for Hamiltonian systems in canonical coordinates, but not necessarily in non-canonical coordinates. Applying the change of coordinates $\etab = \Psi_t(\etab_0)$ in the conservation of mass and using the property \eqref{dot:DPsi} yields

$$
\be
\begin{aligned}
  0 &= \dt{} \int_{\Vcal_0} f_1(t,\Psi_t(\etab_0)) \,|\tn{det}D\Psi_t|(\etab_0)\,\tn d\etab_0 
  \\[1mm]
  & = \int_{\Vcal_0} \Big[ (\pa_t f_1 + \dot \Psi_t(\etab_0) \cdot \nabla f_1) \,|D\Psi_t|(\etab_0) + f_1\,\dt{} |\tn{det}(D\Psi_t)| \Big]\,\tn d\etab_0 
  \\[1mm]
  & = \int_{\Vcal_0} \Big[ \pa_t f_1 + \Gb \cdot \nabla f_1 + f_1\,\nabla \cdot \Gb \Big] |\tn{det}(D\Psi_t)|\,\tn d\etab_0 \,.
 \end{aligned}
\ee \label{int1}
$$

Since the volume $\Vcal_0$ is arbitrary, this implies the PDE 

$$
\be
 \pa_t f_1 + \nabla \cdot (\Gb\,f_1) = 0\,.
\ee \label{eq:f1}
$$

Substituting $f_1 = \sqrt g f$ and moreover using the Liouville property \eqref{Liouville} leads to the Vlasov equation in logical coordinates,

$$
\be
 \pa_t f + \Gb \cdot \nabla f = 0\,.
\ee \label{eq:f}
$$

On the other hand, starting from the Euler-Lagrange map \eqref{ELmap} and making the change of coordinates $\xib = \Psi_t(\etab_0)$, we compute

$$
\be
\begin{aligned}
 f_1(t,\etab) &= \int_{\RR^{2n}} \delta\big(\etab - \Psi_t(\etab_0) \big)\, f_1(0,\etab_0) \,\tn d \etab_0
 \\[1mm]
 &= \int_{\RR^{2n}} \delta\big(\etab - \xib \big)\, f_1(0,\Psi_t^{-1}(\xib))\,|\tn{det}\,D\Psi_t^{-1}|(\xib) \,\tn d \xib
 \\[1mm]
 &= \int_{\RR^{2n}} \delta\big(\etab - \xib \big)\, \frac{f_1(0,\Psi_t^{-1}(\xib))}{|\tn{det}\,D\Psi_t|(\Psi_t^{-1}\xib)} \,\tn d \xib
 \\[1mm]
 &= \frac{f_1(0,\Psi_t^{-1}(\etab))}{|\tn{det}\,D\Psi_t|(\Psi_t^{-1}\etab)} \,.
\end{aligned}
\ee 
$$

Evaluating this along a characteristic $\etab = \Psi_t(\etab_0)$ yields

$$
\be
 f_1(t,\Psi_t(\etab_0))\,|\tn{det}\,D\Psi_t|(\etab_0) = f_1(0,\etab_0) = const.
\ee
$$

This statement is equivalent to \eqref{int1}, leading to the equations \eqref{eq:f1} and \eqref{eq:f} by taking the time derivative.

Coming back to our problem at hand, we observe that the characteristics \eqref{chars} are canonical with Hamiltonian $H= v^2/2 - \phi$ where $E = -\pa_x \phi$. In logical coordinates they are non-canonical and read

$$
\be
\left\{
\begin{aligned}
 \dt{\eta_k(t)} &= \frac{v_k(t)}{\sqrt g(\eta_k(t))}\,,\qquad && \eta_k(0) = \eta_{k0}\,,
 \\[1mm]
 \dt{v_k(t)} &= - E(t,\eta_k(t)) \,,\qquad && v_k(0) = v_{k0}\,,
\end{aligned}
\right.
\ee \label{chars:logic}
$$

where $\sqrt g = F'$. This system falls into the framework \eqref{sys:etab}. Hence, the Vlasov equation in logical space reads

$$
\be
 \pa_t\,f + \frac{v}{\sqrt g}\,\pa_\eta\, f - E\,\pa_v\, f = 0\,,
\ee \label{Vlasov:logic}
$$

and $f(t,\eta,v)$ is conserved along the characteristics $(\eta(t),v(t))$, solutions of \eqref{chars:logic}. Besides we note that the Jacobian  $\sqrt g = F'$ is NOT conserved along $(\eta(t),v(t))$, but rather satisfies the Liouville property \eqref{Liouville}, with $\nabla\cdot \Gb \neq 0$. The pseudo-distribution function $f_1 = \sqrt g f$ satisfies \eqref{eq:f1} which in our case reads

$$
\be
 \pa_t\, f_1 + \pa_\eta \Big(\frac{v}{\sqrt g}\,f_1\Big) - \pa_v (E\,f_1 ) = 0\,.
\ee
$$


### Expectation value <a class="anchor" id="expect"></a>

On the logical domain, the expectation value of a random variable $h(H)$, where $H$ (capital $\eta$) is  a continuous random variable on $[0,1]$, is given by 

$$
\be
 \EE[h(H)] = \int_0^1 h(\eta)\,s(\eta)\sqrt g\,\tn d \eta\,,\qquad \int_0^1 s(\eta)\sqrt g\,\tn d \eta = 1\,.
\ee
$$

The corresponding estimator from samples $(\eta_k)_{k=0}^{K-1}$ reads

$$
\be
 \EE[h(H)]\approx \hat \EE = \frac 1K \sum_{k=0}^{K-1} h(\eta_k)\,.
\ee
$$

Hence, the __PDF in logical space__ is the 1-form $s_1 = s\,\sqrt g$ (in 3D it is a 3-form).
When drawing markers in the logical domain, we need to draw according to $s_1$. However, $s_1$ is not conserved along the characteristics, but $s = s_1/\sqrt{g}$ is. Therefore the computation of the weights goes as follows, starting from equation \eqref{j}:

$$
\be
\begin{aligned}
 \int_0^L\int_\RR v f\,Q\,\tn d v\,\tn d x &= \int_0^1\int_\RR v f\,Q\,\sqrt g\,\tn d v\,\tn d \eta
 \\[1mm]
 &= \int_0^1\int_\RR v\, \underbrace{\frac{f(t,\eta,v)}{s(t,\eta,v)}}_{=:w(t,\eta,v)}\,Q(\eta)\,\underbrace{s(t,\eta,v)\,\sqrt g}_{=s_1}\,\tn d v\,\tn d \eta 
 \\[1mm]
 &= \EE[V w(t,H,V) Q(H)]\,,
\end{aligned}
\ee \label{j:logic}
$$

where $(H,V)$ are random variables ($H$ is the capital $\eta$), distributed according to $s_1$. The function $s$ solves the Vlasov equation \eqref{Vlasov:logic} and thus the weights $w_{0k} = f_0(\eta_{k0},v_{k0})/s_0(\eta_{k0},v_{k0})$ are constant in time.



## Numerical scheme <a class="anchor" id="scheme"></a>

### Semi-discrete in space (continuous in time) <a class="anchor" id="semi"></a>

Regarding Ampère's law, the question is whether we should choose the weak formulation \eqref{weak:0} in terms of 0-forms or the weak formulation \eqref{weak:1} in terms of 1-forms. The answer is given by the charge conservation law,

$$
\be
 \pa_t \int_\RR f \,\tn d v + \frac{1}{\sqrt g} \pa_\eta \int_\RR v\,f\,\tn d v = 0\,,
\ee \label{charge}
$$

which is implied by the Vlasov equation. In a structure-preserving method we want some discrete version of \eqref{charge} to be satisfied exactly. The current being obtained from the weak Ampère's law, the charge must be obtained from a weak Gauss law. For this we choose $E$ to be a 1-form and the charge density $\varrho$ as a 0-form

$$
\be
 *\dd *E^1 = \varrho^0 = \left( 1 - \int_\RR v\, f\,\tn d v \right)^0 \,.
\ee
$$

Then we multiply by a test function $\alpha^0\lra a \in H^1([0,1])$ and apply the co-differential operator to move the derivative to the test function to have a weak formulation:

$$
\begin{alignat}{2}
 &(*\dd *E^1, \alpha^0) &&= (\varrho^0, \alpha^0) \qquad \forall \ \alpha^0\,,
 \\[2mm]
 \Leftrightarrow\quad  &-(E^1, \dd \alpha^0) &&= (\varrho^0, \alpha^0) \qquad \forall \ \alpha^0\,, 
 \\[1mm]
 \Leftrightarrow\quad &- \int_0^1 E_1\,a'\,\frac{1}{\sqrt g}\,\tn d \eta &&= \int_0^1 \left( 1 - \int_\RR v\, f\,\tn d v \right) a\,\sqrt g \,\tn d \eta \qquad \forall\ a\in H^1([0,1]) \,.  \label{Gauss:logic}
\end{alignat}
$$

If $E$ was chosen as a 0-form, Gauss law would either hold strongly, violating discrete charge conservation in PIC, 

$$
\be
 \dd E^0 = \varrho^1\,,
\ee
$$

or weakly, but with the Hodge operator appearing:

$$
\begin{align}
 &(*\dd **E^0, \alpha^0) = (\varrho^0, \alpha^0) \qquad \forall \ \alpha^0\,,
 \\[1mm]
 \Leftrightarrow\quad & -(*E^0, \dd \alpha^0) = (\varrho^0, \alpha^0) \qquad \forall \ \alpha^0 \,.
\end{align}
$$

The discretization of the Hodge is ambiguous; we would have to choose it such that charge conservation holds. But this effort is not necessary when we choose $E$ as a 1-form and $\varrho$ as a 0-form. The general rule for structure-preserving codes with STRUPHY is: __always avoid the Hodge operator!__

We thus expand the discrete electric field $E_h \in V_1$ and the test functions $Q_h \in V_1$ in the $D$-spline basis of degree $p$ (these are $N$-splines of degree $p-1$ times $p/h$, see `HOWTO_use_module_bsplines.ipynb`),

$$
\be
 E_h(t,\eta) = \sum_i e_i(t)\,D^p_i(\eta)\,,\qquad Q_h(\eta) = \sum_i q_i\,D^p_i(\eta)\,,
\ee
$$

and denote the coefficient vectors by $\eb = (e_i)_{i=0}^{N-1} \in \RR^N$ and $\qb = (q_i)_{i=0}^{N-1} \in \RR^N$. The mass matrix in the $V_0$ space is 

$$
\be
 \MM_1 = (m^1_{i,j})_{i,j} \in \RR^{N\times N}\,,\qquad m^1_{i,j} = \int_0^1 D^p_i\,D^p_j \frac{1}{\sqrt g}\,\tn d \eta\,.
\ee
$$

The semi-discrete version of the weak equation \eqref{weak:1} then reads

$$
\be
 \dt \eb^\top \MM_1 \qb = \jb^\top \qb\,,\qquad \forall\ \qb \in \RR^N\,, 
 \ee \label{Amp:semi}
$$

where 

$$
\be
 \jb = (j_i)_{i=0}^{N-1}\in \RR^N\,,\qquad j_i = \left( \int_\RR v\,f\,\tn d v, D^p_i \right)_{L^2}\,.
\ee
$$

The coefficients $j_i$ are approximate via **Monte-Carlo integration**, hence

$$
\be
\begin{aligned}
 \qquad j_i &= \left( \int_\RR v \, f\,\tn d v, D^p_i \right)_{L^2}
 \\[1mm]
 &= \int_0^1 \int_\RR v\, \frac fs\,\frac{1}{\sqrt g} D^p_i\,\underbrace{s\sqrt g}_{=s_1}\,\tn d v\, \tn d \eta 
 \\[0mm]
 &= \EE\left[V\,w(H,V)\,\frac{1}{\sqrt g (H)}\,D^p_i(H)\right] 
 \\[1mm]
 &\approx \frac 1K \sum_k w_{0k}\, v_k(t)\,\frac{1}{\sqrt g (\eta_k(t))}\,D^p_i(\eta_k(t))\,,
 \end{aligned}
\ee
$$

where $(H,V)$ are random variables ($H$ is the capital $\eta$). Here, $s$ solves the Vlasov equation \eqref{Vlasov:logic} and the weights are $w_{0k} = f_0(\eta_{k0},v_{k0})/s_0(\eta_{k0},v_{k0})$, where $s_0 = s_1/\sqrt g$ and $s_1$ is the PDF according to which the initial markers are drawn. The characteristics $(\eta_k(t),v_k(t))$ are computed from

$$
\be
\left\{
\begin{aligned}
 \dt{\eta_k(t)} &= \frac{v_k(t)}{\sqrt g(\eta_k(t))}\,,\qquad && \eta_k(0) = \eta_{k0}\,,
 \\[1mm]
 \dt{v_k(t)} &= - \frac{1}{\sqrt g(\eta_k(t))}\sum_i e_i(t)\,D^p_i(\eta_k(t)) \,,\qquad && v_k(0) = v_{k0}\,,
\end{aligned}
\right.
\ee \label{chars:num}
$$

Here, the factor $1/\sqrt g$ in the velocity equation stems from the push-forward of 1-forms. Introducing 

$$
\begin{align}
 \etab(t) &= (\eta_k(t))_{k=0}^{K-1} \in \RR^K\,,\qquad \vb(t) = (v_k(t))_{k=0}^{K-1} \in \RR^K\,,
 \\[2mm]
 \Wcal &= \frac 1K \tn{diag}[(w_{0k})_{k=0}^{K-1}] \in \RR^{K\times K}\,,
 \\[2mm]
 \Gcal(\etab) &= \tn{diag}[(\sqrt g (\eta_k))_{k=0}^{K-1}] \in \RR^{K\times K}\,,
 \\[2mm]
 \Dcal(\etab) &= (d_{i,k})_{i,k = 0,0}^{N-1,K-1} \in \RR^{N\times K}\,,\qquad d_{i,k} = D^p_i(\eta_k)\,,
\end{align}
$$

we can write the system \eqref{Amp:semi}-\eqref{chars:num} as

$$
\be
\left\{
\begin{aligned}
 \MM_1 \dot \eb &= \Dcal(\etab) \Gcal^{-1} \Wcal\,\vb\,,
 \\[2mm]
  \dot \etab &= \Gcal^{-1}(\etab)\,\vb\,,
 \\[2mm]
  \dot \vb &= - \Gcal^{-1}\Dcal^\top(\etab) \,\eb\,.
\end{aligned}
\right.
\ee \label{sys:semi}
$$

The energy \eqref{VA:en} needs to be translated to the logical domain and then discretized:

$$
\be
\begin{aligned}
 H &= \frac 12 \int_0^L\int_\RR |v|^2\,f\,\tn d v\,\tn d x + \frac 12 \int_0^L |E|^2\,\tn d x
 \\[1mm]
 &= \frac 12 \int_0^1\int_\RR |v|^2\,f\,\sqrt g\,\tn d v\,\tn d \eta + \frac 12 \int_0^1 |\sqrt g\, E|^2\,\frac{1}{\sqrt g}\,\tn d \eta\,.
 \end{aligned}
\ee \label{H2}
$$

This can be approximated by

$$
\be
\begin{aligned}
 H_h(\eb,\etab,\vb) &= \frac 12\,\frac 1K \sum_{k=0}^{K-1} w_{0k}\,|v_k(t)|^2 + \frac 12 \, \eb^\top \MM_1 \eb
 \\[1mm]
 &= \frac 12\, \vb^\top \Wcal\,  \vb + \frac 12 \, \eb^\top \MM_1 \eb \,.
 \end{aligned}
\ee \label{H:disc}
$$

The gradient of this Hamiltonian reads

$$
\be
 \nabla H_h = \begin{pmatrix}
  \MM_1 \eb
  \\[1mm]
  0
  \\[1mm]
  \Wcal\,  \vb
 \end{pmatrix}
\ee
$$

We thus write the system \eqref{sys:semi} as

$$
\be
\begin{aligned}
 \begin{pmatrix}
  \dot \eb 
 \\[1mm]
  \dot \etab 
 \\[1mm]
  \dot \vb 
  \end{pmatrix}
  =
  \underbrace{
  \begin{pmatrix}
  0 & 0 & \MM_1^{-1} \Dcal(\etab) \Gcal^{-1}
  \\[1mm]
  0 & 0 & \Gcal^{-1}\Wcal^{-1}
  \\[1mm]
  - \Gcal^{-1} \Dcal^\top(\etab) \MM_1^{-1} & -\Wcal^{-1}\Gcal^{-1} & 0
  \end{pmatrix}
  }_{= J}
  \underbrace{
  \begin{pmatrix}
  \MM_1 \eb
  \\[1mm]
  0
  \\[1mm]
  \Wcal\, \vb
 \end{pmatrix}
  }_{= \nabla H_h}
\end{aligned}
\ee \label{sys:Ham}
$$

The matrix $J\in \RR^{(N+2K) \times (N+2K)}$ is clearly skew-symmetric, $J^\top = -J$, and thus the discrete Hamiltonian $H_h$ is conserved exactly by this system of ODEs:

$$
\be
 \dot H_h(\eb,\etab,\vb) = \nabla H_h^\top (\dot\eb,\dot\etab,\dot\vb) = \nabla H_h^\top J\, \nabla H_h = -\nabla H_h^\top J\, \nabla H_h = 0\,. 
\ee
$$


### Conservation of Gauss' law and initialization <a class="anchor" id="gauss"></a>

Let us discretize Gauss' law \eqref{Gauss:logic}. For this we expand the test functions $a_h \in V_0$ in the $N$-spline basis of degree $p$,

$$
\be
 a_h(t,\eta) = \sum_{i=0}^{N-1} a_i(t)\,N^p_i(\eta)\,,
\ee
$$

and denote the coefficient vectors by $\ab = (a_i)_{i=0}^{N-1} \in \RR^N$. We thus obtain

$$
\begin{alignat}{1}
 - \eb^\top \MM_1 \GG\, \ab &= \Cb^\top \ab - \wb_0^\top \Ncal^\top(\etab)\, \ab \qquad \forall \ \ab \in \RR^N\,,
 \\[1mm]
 \Leftrightarrow\qquad  -\GG^\top \MM_1 \eb &= \Cb - \Ncal(\etab) \wb_0\,. \label{Gauss:disc}
\end{alignat}
$$

where we introduced the gradient matrix $\GG$ (see `HOWTO_use_module_bsplines.ipynb`), as well as

$$
\begin{align}
 \Cb &= (c_i)_{i=0}^{N-1} \in \RR^N\,,\qquad c_i = \int_0^1 N_i^p\,\sqrt g\,\tn d \eta\,,
\\[0mm]
\wb_0 & = \frac 1K (w_{0k})_{k=0}^{K-1} \in \RR^K\,,
\\[2mm]
 \Ncal(\etab) &= (n_{i,k})_{i,k = 0,0}^{N-1,K-1} \in \RR^{N\times K}\,,\qquad n_{i,k} = N^p_i(\eta_k)\,.
\end{align}
$$

Taking the time derivative in \eqref{Gauss:disc} and using the important B-spline property for the derivatives (see `HOWTO_use_module_bsplines.ipynb`),

$$
\be
 \dt{} N_i^p(\eta_k) =  (N_i^p)'(\eta_k)\,\dot \eta_k = (D_{i-1}^p - D_i^p)(\eta_k)\,\frac{v_k}{\sqrt g(\eta_k)}\,,
\ee
$$

yields

$$
\be
 \dt{}(\Cb - \Ncal(\etab) \wb_0) = -\GG^\top \Dcal \Gcal^{-1} \Wcal \,\vb\,.
\ee
$$

On the other hand, by multiplying Gauss law with $\GG^\top$ from the left we obtain

$$
\be
 \dt{}(\GG^\top \MM_1 \eb) = \GG^\top \Dcal \Gcal^{-1} \Wcal \,\vb = -\dt{}(\Cb - \Ncal(\etab) \wb_0)\,.
\ee
$$

This means that if we solve Vlasov-Ampère with our discretization, we have

$$
\be
 \dt{}(\GG^\top \MM_1 \eb + \Cb - \Ncal(\etab) \wb_0) = 0\,.
\ee
$$

Hence if we can initialize our variables such that 

$$
\be
 \GG^\top \MM_1 \eb + \Cb - \Ncal(\etab) \wb_0 = 0
\ee
$$

is fulfilled at $t=0$, then it will remain fulfilled during the simulation for all $t>0$. This mimics the behaviour on the continuous level and leads to increased stability.

Ideally, we would draw markers according to the PDF $s_1$, then compute $\Ncal(\etab) \wb_0$ and initialize $\eb$ according to

$$
\be
  \eb =  (\GG^\top \MM_1)^{-1}(- \Cb + \Ncal(\etab) \wb_0)\,.
\ee
$$

Note that $\GG$ is not invertible for periodic boundary conditions and needs to be regularized; this can be done by replacing one line of $\GG$ with the zero-average constraint on the solution, as we would do for solving the Poisson equation.


### Time discretization <a class="anchor" id="time"></a>

The system \eqref{sys:Ham} can be advanced in time either with Hamiltonian splitting or with Poisson splitting. Because Hamiltonian splitting keeps the Poisson matrix intact, it keeps the exact Gauss law with oscillating energy. By contrast, Poisson splitting features the exact Hamiltonian, leading to exact energy conservation.

__Hamiltonian splitting:__

We split the Hamiltonian as $H_h = H_1 + H_2$ with 

$$
\be
H_1 = \frac 12\, \vb^\top \Wcal\,  \vb\,,\qquad H_2 = \frac 12 \, \eb^\top \MM_1 \eb \,,
\ee 
$$

and solve the two subsystems

$$
\be
\begin{aligned}
  \tn{Substep\ 1:}\qquad
 \begin{pmatrix}
  \dot \eb 
 \\[1mm]
  \dot \etab 
 \\[1mm]
  \dot \vb 
  \end{pmatrix}
  = J \nabla H_1
  =
  \begin{pmatrix}
  0 & 0 & \MM_1^{-1} \Dcal(\etab) \Gcal^{-1}
  \\[1mm]
  0 & 0 & \Gcal^{-1}\Wcal^{-1}
  \\[1mm]
  - \Gcal^{-1} \Dcal^\top(\etab) \MM_1^{-1} & -\Wcal^{-1}\Gcal^{-1} & 0
  \end{pmatrix}
   \begin{pmatrix}
  0
  \\[1mm]
  0
  \\[1mm]
  \Wcal\, \vb
 \end{pmatrix}
\end{aligned}
\ee 
$$

$$
\be
\begin{aligned}
  \tn{Substep\ 2:}\qquad
 \begin{pmatrix}
  \dot \eb 
 \\[1mm]
  \dot \etab 
 \\[1mm]
  \dot \vb 
  \end{pmatrix}
  = J \nabla H_2
  =
  \begin{pmatrix}
  0 & 0 & \MM_1^{-1} \Dcal(\etab) \Gcal^{-1}
  \\[1mm]
  0 & 0 & \Gcal^{-1}\Wcal^{-1}
  \\[1mm]
  - \Gcal^{-1} \Dcal^\top(\etab) \MM_1^{-1} & -\Wcal^{-1}\Gcal^{-1} & 0
  \end{pmatrix}
   \begin{pmatrix}
  \MM_1 \eb
  \\[1mm]
  0
  \\[1mm]
  0
 \end{pmatrix}\,.
\end{aligned}
\ee 
$$

In Substep 1 we need to solve

$$
\be
\begin{aligned}
  \tn{Substep\ 1:}\qquad
 \begin{pmatrix}
  \dot \eb 
 \\[1mm]
  \dot \etab 
 \\[1mm]
  \dot \vb 
  \end{pmatrix}
  = J \nabla H_1
  =
  \begin{pmatrix}
  \MM_1^{-1} \Dcal(\etab) \Gcal^{-1} \Wcal\, \vb
  \\[1mm]
  \Gcal^{-1}\vb
  \\[1mm]
  0
  \end{pmatrix}
\end{aligned}
\ee 
$$

with initial conditions $(\eb^n,\etab^n,\vb^n)$. This yields

$$
\be
\begin{aligned}
 \begin{pmatrix}
   \eb(t) 
 \\[1mm]
   \etab(t) 
 \\[1mm]
   \vb(t) 
  \end{pmatrix}
  =
  \begin{pmatrix}
  \eb^n + \MM_1^{-1} \int_{t^n}^t\Dcal(\etab(t'))\,\tn dt'\, \Gcal^{-1} \Wcal\, \vb^n
  \\[1mm]
  \etab^n + \Gcal^{-1}\vb^n\,t
  \\[1mm]
  \vb^n
  \end{pmatrix}\,.
\end{aligned}
\ee 
$$

The time integrals in $\eb(t)$ can be computed exactly, as the integrands are piece-wise polynomials in $t'$. However, the implementation can be cumbersome.

In Substep 2 we need to solve

$$
\be
\begin{aligned}
  \tn{Substep\ 2:}\qquad
 \begin{pmatrix}
  \dot \eb 
 \\[1mm]
  \dot \etab 
 \\[1mm]
  \dot \vb 
  \end{pmatrix}
  = J \nabla H_2
  =
  \begin{pmatrix}
  0 
  \\[1mm]
  0 
  \\[1mm]
  - \Gcal^{-1} \Dcal^\top(\etab) \eb 
  \end{pmatrix}\,.
\end{aligned}
\ee 
$$

with initial conditions $(\eb^n,\etab^n,\vb^n)$. This yields

$$
\be
\begin{aligned}
 \begin{pmatrix}
   \eb(t) 
 \\[1mm]
   \etab(t) 
 \\[1mm]
   \vb(t) 
  \end{pmatrix}
  =
  \begin{pmatrix}
  \eb^n 
  \\[1mm]
  \etab^n 
  \\[1mm]
  \vb^n + - \Gcal^{-1} \Dcal^\top(\etab^n) \eb^n \,t
  \end{pmatrix}\,.
\end{aligned}
\ee 
$$

This step is thus very simple. Let us denote the flow maps of the two substeps by $\Phi_t^1$ and $\Phi_t^2$, respectively, such that

$$
\be
 \begin{pmatrix}
   \eb(t) 
 \\[1mm]
   \etab(t) 
 \\[1mm]
   \vb(t) 
  \end{pmatrix} = \Phi_t^1
  \begin{pmatrix}
   \eb^n
 \\[1mm]
   \etab^n 
 \\[1mm]
   \vb^n
  \end{pmatrix}\,,\qquad  \begin{pmatrix}
   \eb(t) 
 \\[1mm]
   \etab(t) 
 \\[1mm]
   \vb(t) 
  \end{pmatrix} = \Phi_t^2
  \begin{pmatrix}
   \eb^n
 \\[1mm]
   \etab^n 
 \\[1mm]
   \vb^n
  \end{pmatrix}\,.
\ee
$$

From this we can build splitting methods of arbitrary order, for instance

$$
\be
 \tn{Lie-Trotter\ (1st\ order):} \quad \Phi_{\Delta t} = \Phi_{\Delta t}^2 \Phi_{\Delta t}^1\,,\qquad \tn{Strang\ (2nd\ order):} \quad \Phi_{\Delta t} = \Phi_{\Delta t/2}^2 \Phi_{\Delta t}^1 \Phi_{\Delta t/2}^2\,.
\ee
$$

__Poisson splitting:__

We split the Poisson matrix $J$ into two skew-symmetric parts,

$$
\be
 J_1 = \begin{pmatrix}
  0 & 0 & \MM_1^{-1} \Dcal(\etab) \Gcal^{-1}
  \\[1mm]
  0 & 0 & 0
  \\[1mm]
  - \Gcal^{-1} \Dcal^\top(\etab) \MM_1^{-1} & 0 & 0
  \end{pmatrix}\,,\qquad
 J_2 = \begin{pmatrix}
  0 & 0 & 0
  \\[1mm]
  0 & 0 & \Gcal^{-1}\Wcal^{-1}
  \\[1mm]
  0 & -\Wcal^{-1}\Gcal^{-1} & 0
  \end{pmatrix}\,,
\ee
$$

and solve the two subsystems

$$
\be
\begin{aligned}
  \tn{Substep\ 1:}\qquad
 \begin{pmatrix}
  \dot \eb 
 \\[1mm]
  \dot \etab 
 \\[1mm]
  \dot \vb 
  \end{pmatrix}
  = J_1 \nabla H_h
  =
  \begin{pmatrix}
  0 & 0 & \MM_1^{-1} \Dcal(\etab) \Gcal^{-1}
  \\[1mm]
  0 & 0 & 0
  \\[1mm]
  - \Gcal^{-1} \Dcal^\top(\etab) \MM_1^{-1} & 0 & 0
  \end{pmatrix}
   \begin{pmatrix}
  \MM_1\eb
  \\[1mm]
  0
  \\[1mm]
  \Wcal\, \vb
 \end{pmatrix}
\end{aligned}
\ee 
$$

$$
\be
\begin{aligned}
  \tn{Substep\ 2:}\qquad
 \begin{pmatrix}
  \dot \eb 
 \\[1mm]
  \dot \etab 
 \\[1mm]
  \dot \vb 
  \end{pmatrix}
  = J_2 \nabla H_h
  =
  \begin{pmatrix}
  0 & 0 & 0
  \\[1mm]
  0 & 0 & \Gcal^{-1}\Wcal^{-1}
  \\[1mm]
  0 & -\Wcal^{-1}\Gcal^{-1} & 0
  \end{pmatrix}
   \begin{pmatrix}
  \MM_1\eb
  \\[1mm]
  0
  \\[1mm]
  \Wcal\, \vb
 \end{pmatrix}\,,
\end{aligned}
\ee 
$$

with initial conditions $(\eb^n,\etab^n,\vb^n)$.

Substep 1 can be solved with a Crank-Nicolson (CN) method which is implicit and energy conserving. For this let us write $\zb = (\eb,\vb)$ and introduce the matrix

$$
\be
 A := \begin{pmatrix}
  0 & \MM_1^{-1} \Dcal(\etab^n) \Gcal^{-1} \Wcal
  \\[1mm]
  - \Gcal^{-1} \Dcal^\top(\etab^n) & 0 
  \end{pmatrix} = \begin{pmatrix}
  0 & A_1
  \\[1mm]
  A_2 & 0 
  \end{pmatrix} \in \RR^{(N+K) \times (N+K)}\,.
\ee
$$

Note that $\etab(t) = \etab^n$ is constant during this substep. The CN scheme reads

$$
\be
 \frac{\zb^{n+1} - \zb^n}{\Delta t} = A \frac{\zb^{n+1} + \zb^n}{2}\,.
\ee
$$

Because the particle number $K>>1$ can be very large, we cannot invert the matrix $A$ directly. Rather, we use the following trick: first, let us rewrite the system as

$$
\begin{align}
 \frac{\eb^{n+1} - \eb^n}{\Delta t} &= \MM_1^{-1} \Dcal(\etab) \Gcal^{-1} \Wcal\, \frac{\vb^{n+1} + \vb^n}{2}\,,
 \\[1mm]
 \frac{\vb^{n+1} - \vb^n}{\Delta t} &= - \Gcal^{-1} \Dcal^\top(\etab) \frac{\eb^{n+1} + \eb^n}{2}\,, \label{sys:v}
\end{align}
$$

Then we substitute $\vb^{n+1}$ from the second equation into the first equation,

$$
\be 
 \frac{\eb^{n+1} - \eb^n}{\Delta t} = \underbrace{\MM_1^{-1} \Dcal(\etab) \Gcal^{-1} \Wcal\, \left( - \frac{\Delta t}{2}\Gcal^{-1} \Dcal^\top(\etab) \right)}_{=:S \in \RR^{N\times N}} \frac{\eb^{n+1} + \eb^n}{2} + \MM_1^{-1} \Dcal(\etab) \Gcal^{-1} \Wcal\,\vb^n \,.
\ee \label{sys:e}
$$

The matrix $S \in \RR^{N\times N}$ is relatively small and does not pose problems regarding memory. We can thus assemble and invert it (in sparse format). The assembly involves a loop over the particles. Once we have solved \eqref{sys:S} for $\eb^{n+1}$ we can obtain the new particle velocities from \eqref{sys:v}, by simply looping over the particles. In this way we avoid having to deal with large matrices.



### Old stuff

The solution reads

$$
\be
 \zb^{n+1} = (\mathbb 1 - \Delta t/2\, A)^{-1}(\mathbb 1 + \Delta t/2\, A)\zb^{n}\,,
\ee
$$

The inverse of the block matrix can be written as

$$
\be
\begin{aligned}
 (\mathbb 1 - \Delta t/2\, A)^{-1} &= \begin{pmatrix}
  \mathbb 1_N & -\Delta t /2\,A_1
  \\[1mm]
  - \Delta t /2\,A_2 & \mathbb 1_K 
  \end{pmatrix}^{-1} 
  \\[1mm]
  &= \begin{pmatrix}
  S^{-1}  & \Delta t /2\,S^{-1} A_1 
  \\[1mm]
  \Delta t /2\,A_2 S^{-1} & \mathbb 1_K + \Delta t^2 /4\,A_2 S^{-1} A_1
  \end{pmatrix}\,,
\end{aligned}
\ee
$$

where $S = \mathbb 1_N - \Delta t^2 /4\,A_1 A_2$ is the __Schur complement__ of $\mathbb 1_K$ in $A$. Now to write the solution $\zb^{n+1}$ you only need the $LU$-decomposition of the $N\times N$ matrix $S$, and then proceed with matrix-vector multiplication. Substep 2 is trivial.